In [1]:
from crawling import *
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from pymongo import MongoClient
# need dnspython, PyOpenSSL, requests and service_identity for srv and ocsp
from urllib import parse
from naverapi import *

In [2]:
client = MongoClient(
    "mongodb+srv://cluster0.c6ccx.mongodb.net",
    authsource="$external",
    authmechanism="MONGODB-X509",
    tls=True,
    tlsCertificateKeyFile="../../../mongodb.pem",
)
db = client["webtoon"]
collection = db["data"]
collection_backup = db["backup"]
collection_backup2 = db["backup2"]

In [ ]:
dateFormat = "%Y-%m-%dT%H:%M:%S%z"
data = list(collection.find())[:10]
bar = ProgressBar(sum(len(i["episode"]) for i in data))
for document in data:
    for i, episode in enumerate(document["episode"]):
        bar.next()
        target_url = episode["url"]
        pagenum = getJson(target_url)["result"]["pageModel"]["lastPage"]
        if i == len(document["episode"])-1:
            end_date = datetime.datetime(2022,1,1).replace(tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))
        else:
            end_date = document["episode"][i+1]["date"]
        wanted_key = ["idNo", "idProvider", "contents", "country", "regTime", "sympathyCount", "antipathyCount", "hiddenByCleanbot"]
        # add best comment
        best_comment_json = getJson(target_url, {"sort":"BEST"})["result"]["commentList"]
        best_comment_data = []
        for best_comment in best_comment_json:
            best_comment_data.append(dict_filter(comment_json, wanted_key))
        collection.update_one(
            {"title": document["title"], "episode.no":episode["no"]},
            {
                "$set":{
                    "episode.$.comment.best":comment_data
                }
            }
        )
        # add new comment
        break_seq = False
        comment_data = []
        while pagenum > 0 and not break_seq:
            comment_json = getJson(target_url, {"page":pagenum})
            comment_list = comment_json["result"]["commentList"][::-1]
            for comment in comment_list:
                if datetime.datetime.strptime(comment["regTime"], dateFormat) >= end_date:
                    break_seq = True
                    break
                comment_data.append(dict_filter(comment_json, wanted_key))
            pagenum -= 1
        collection.update_one(
            {"title": document["title"], "episode.no":episode["no"]},
            {
                "$set":{
                    "episode.$.comment.new":comment_data
                }
            }
        )